In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
notes = notes[['Category','Categorization','Word','Definition']].copy()

definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category'].isin(['Machine Learning','Visualizations'])]

from df_processing import final_dataset_for_markdown
d_learning_notes = final_dataset_for_markdown(notes,definitions)
d_learning_notes

,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
1,Machine Learning,Definition,"Bias, Fairness and Ethics",This step examines whether the model produces ...
2,Machine Learning,Definition,Data Collection,"Data collection involves identifying, sourcing..."
3,Machine Learning,Definition,Data Preparation,Data preparation focuses on cleaning and struc...
4,Machine Learning,Definition,Deployment,Deployment is the process of integrating the t...
...,...,...,...,...
384,Mathematics,Best Linear Unbiased Estimator,Requirements,Homoscedasticity: Variance of the residuals sh...
385,Mathematics,Best Linear Unbiased Estimator,Requirements,Independence: Observations should be independe...
386,Mathematics,Best Linear Unbiased Estimator,Requirements,Linearity: Relationship between Independent an...
387,Mathematics,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity: No perfect linear rel...


In [8]:

def final_dataset_for_markdown_V3(notes=None,
                                  definitions=None,
                                  export_location='/Users/derekdewald/Documents/Python/Github_Repo/Streamlit/DataDictionary/'):
    
    '''
    
    Function which helps to combined Notes and Definitions into a Single Combined Representation which can ultimately be used as a Learning Reference Tools.

    How this function Works:
    It takes the two sheets and attemps to Consolidate them together to make Final Dataset, generated as d_learning_notes.csv.

    Approach is to Take Notes As the Framework and Distribute All Information into the Notes Sequentially in Specific Order so i can 
    Understand the structure and how to continue and utilize the Sheet.

    Principles: 
        - Define all types of Records in Notes and then Move them Out Definiton By Definition
        - Worry about Order at End.

    Step 1: Insert Records where Categorization = Definition directly into Sheet.
            With Modification:
                Where Word is also A Categorization, Update Categorization from Definition to Word, Change Word to Definition, 
                and Updated Definition to include WORD:
            
    Parameters:
        notes(df): DataFrame of D Notes as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv
        definitions(df): DataFrame of D Definitions as stored in: https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv
        export_location(str): Location of where to Save CSV File. If blank, no CSV is made.

    date_created:20-Dec-25
    date_last_modified: 21-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        final_dataset_for_markdown()

    ##############

    Has been tested for a Single Value - Machine Learning. Need to Validate once extending.

    ##############

    
    '''

    from data_d_dicts import links

    try:
        len(notes)
    except:
        notes = pd.read_csv(links['google_notes_csv'])

    final_df = notes.copy()
    
    try:
        len(definitions)
    except:
        definitions = pd.read_csv(links['google_definition_csv'])

    temp_def = definitions[['Category','Categorization','Word','Definition']].copy()

    # Step 1
    # Create Definition only df and a Residual Definition DF res_def_df
    df_cat_is_definition = temp_def[temp_def['Categorization']=='Definition'].copy()
    cat_list = notes['Categorization'].unique().tolist()
    df_cat_is_definition['Categorization'] = np.where(df_cat_is_definition['Word'].isin(cat_list),df_cat_is_definition['Word'],df_cat_is_definition['Categorization'])

    mask = df_cat_is_definition["Categorization"] != "Definition"

    df_cat_is_definition.loc[mask, "Definition"] = (
        df_cat_is_definition.loc[mask, "Word"].astype(str)
        + ": "
        + df_cat_is_definition.loc[mask, "Definition"].astype(str)
    )

    df_cat_is_definition['Word'] = np.where(df_cat_is_definition['Categorization']!='Definition','Definition',df_cat_is_definition['Word'])
    df_cat_is_definition['ZATION_RANK'] = 0
    res_def_df = temp_def[temp_def['Categorization']!='Definition'].copy()

    # Insert 
    final_df = pd.concat([final_df,df_cat_is_definition])

    # Step 2

    # Ranking for Sort ORder
    rank_df1 = notes.drop_duplicates('Category')[['Category']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank1"})
    rank_df2 = notes.drop_duplicates(['Category','Categorization'])[['Category','Categorization']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank2"})
    rank_df2['Rank2'] = rank_df2['Rank2'] + 1
    rank_df3 = notes.drop_duplicates(['Category','Categorization','Word']).reset_index(drop=True).reset_index().drop('Definition',axis=1).rename(columns={'index':'Rank3'})
    rank_df3['Rank3'] = rank_df3['Rank3'] + 1

    final_df = final_df.merge(rank_df1,on=['Category'],how='left').merge(rank_df2,on=['Category','Categorization'],how='left').merge(rank_df3,on=['Category','Categorization','Word'],how='left')

    #final_df['COUNT'] = final_df.groupby(['Category','Categorization','Word']).transform('size')
    #final_df = final_df[~((final_df['Definition']=='')&(final_df['COUNT']!=1))].drop('COUNT',axis=1).reset_index(drop=True)

    if export_location:
        final_df.to_csv(f"{export_location}d_learning_notes.csv",index=False)
    
    return final_df.sort_values(['ZATION_RANK','Rank1','Rank2','Rank3'])

d = final_dataset_for_markdown_V3(notes,definitions)
d
                      

,Category,Categorization,Word,Definition,ZATION_RANK,Rank1,Rank2,Rank3
146,Machine Learning,Problem Definition,Definition,Problem Definition: Problem definition clearly...,0.0,0.0,1.0,NaN
117,Machine Learning,Data Collection,Definition,Data Collection: Data collection involves iden...,0.0,0.0,2.0,NaN
122,Machine Learning,Feature Engineering,Definition,Feature Engineering: Feature engineering is th...,0.0,0.0,4.0,NaN
123,Machine Learning,Feature Selection,Definition,Feature Selection: Feature selection involves ...,0.0,0.0,5.0,NaN
142,Machine Learning,Model,Definition,Model: The model is the mathematical or algori...,0.0,0.0,6.0,NaN
...,...,...,...,...,...,...,...,...
111,Machine Learning,Monitoring,Algorithm,,NaN,0.0,14.0,110.0
112,Machine Learning,Monitoring,Function,,NaN,0.0,14.0,111.0
113,Machine Learning,Monitoring,Procedure,,NaN,0.0,14.0,112.0
114,Machine Learning,Monitoring,TBD,,NaN,0.0,14.0,113.0


In [6]:
d[d['Categorization']=="Problem Definition"]

,Category,Categorization,Word,Definition,ZATION_RANK,Rank1,Rank2,Rank3
146,Machine Learning,Problem Definition,Definition,Problem Definition: Problem definition clearly...,0.0,0.0,1.0,NaN
0,Machine Learning,Problem Definition,Goal,Need this to Pull from Somewhere Else.,NaN,0.0,1.0,1.0
1,Machine Learning,Problem Definition,Approach,,NaN,0.0,1.0,2.0
2,Machine Learning,Problem Definition,Important to Remeber,,NaN,0.0,1.0,3.0
3,Machine Learning,Problem Definition,Lesson Learnt,,NaN,0.0,1.0,4.0
4,Machine Learning,Problem Definition,Algorithm,,NaN,0.0,1.0,5.0
5,Machine Learning,Problem Definition,Function,,NaN,0.0,1.0,6.0
6,Machine Learning,Problem Definition,Procedure,,NaN,0.0,1.0,7.0
7,Machine Learning,Problem Definition,TBD,,NaN,0.0,1.0,8.0


In [33]:
d[d['Category'].isin(['Machine Learning','Mathematics'])]

,Category,Categorization,Word,Definition,CLASS
5,Machine Learning,Definition,"Bias, Fairness and Ethics",This step examines whether the model produces ...,Definition
6,Machine Learning,Definition,Data Collection,"Data collection involves identifying, sourcing...",Definition
7,Machine Learning,Definition,Data Preparation,Data preparation focuses on cleaning and struc...,Definition
8,Machine Learning,Definition,Deployment,Deployment is the process of integrating the t...,Definition
9,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,Definition
10,Machine Learning,Definition,Evaluation,Evaluation measures how well the final model p...,Definition
11,Machine Learning,Definition,Feature Engineering,Feature engineering is the process of creating...,Definition
12,Machine Learning,Definition,Feature Selection,Feature selection involves choosing a subset o...,Definition
13,Machine Learning,Definition,Hyperparameter Tuning,Hyperparameter tuning involves selecting optim...,Definition
14,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...,Definition


In [ ]:

    # Step 1
    # Create Unique Category DF
    unique_cat_df = notes.drop_duplicates(['Category'])[['Category']]

    # Process temp_def such that 
    step1_df = unique_cat_df.merge(temp_def.drop('Category',axis=1),left_on='Category',right_on='Word',how='left')
    step1_df['Categorization'] = 'Definition'
    step1_df['Word'] = step1_df['Category'].copy()
    step1_df['Definition'] = step1_df['Definition'].fillna('Not Defined')

    # Step 2
    unique_cat_df['Categorization'] = 'Definition'
    step2_df = unique_cat_df.merge(temp_def,on=['Category','Categorization'],how='inner')

    # Step 3
    step3_df = notes.drop_duplicates('Categorization')[['Category','Categorization']].merge(definitions[['Word','Definition']].rename(columns={'Word':'Categorization'}),on='Categorization',how='left')    
    step3_df['Word'] = 'Definition'
    step3_df['Definition'] = step3_df['Definition'].fillna('Not Defined')

    return step3_df
    
    # Step 4
    mod_def = definitions[['Category','Categorization','Word','Definition']].copy()
    mod_def['Definition'] = mod_def['Word'] + ": " + mod_def['Definition']
    mod_def = mod_def.drop('Word',axis=1).reset_index(drop=True)
    mod_def = mod_def.rename(columns={'Categorization':'Word','Category':'Categorization'})
    
    # Merge Notes.Categorization and Notes.Word to Definitions.Category and Definitions.Categorization
    step_4df = notes.drop_duplicates(['Categorization','Word'])[['Category','Categorization','Word']].merge(mod_def,on=['Categorization','Word'],how='inner')
    
    # Can DO ORDER AT THE END. USING THE ORIGINAL DATASET and Category MERGE IN

    final_df = pd.concat([notes,step1_df,step2_df,step3_df,step_4df]).drop_duplicates()

    rank_df1 = notes.drop_duplicates('Category')[['Category']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank1"})
    rank_df2 = notes.drop_duplicates(['Category','Categorization'])[['Category','Categorization']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank2"})
    rank_df2['Rank2'] = rank_df2['Rank2'] + 1
    rank_df3 = notes.drop_duplicates(['Category','Categorization','Word']).reset_index(drop=True).reset_index().drop('Definition',axis=1).rename(columns={'index':'Rank3'})
    rank_df3['Rank3'] = rank_df3['Rank3'] + 1
    final_df = final_df.merge(rank_df1,on=['Category'],how='left').merge(rank_df2,on=['Category','Categorization'],how='left').merge(rank_df3,on=['Category','Categorization','Word'],how='left').fillna(0).sort_values(['Rank1','Rank2','Rank3']).reset_index(drop=True).drop(['Rank1','Rank2','Rank3'],axis=1)

    final_df['COUNT'] = final_df.groupby(['Category','Categorization','Word']).transform('size')
    final_df = final_df[~((final_df['Definition']=='')&(final_df['COUNT']!=1))].drop('COUNT',axis=1).reset_index(drop=True)
    # Need to Delete

    if export_location:
        final_df.to_csv(f"{export_location}d_learning_notes.csv",index=False)
    
    return final_df

In [ ]:
# I want to Distribute the Information From Definitions into Notes, While Maintaining the SIMPLEST Structure Possible.
# So the approach is take the notes Framework. Make it Minimal, and merge Information in.
# Over Engineering the Process.


In [ ]:
# Distribute Definitions.

In [24]:
definitions[definitions['Categorization']=="Definition"][['Category','Categorization','Word','Definition']]



,Category,Categorization,Word,Definition
5,Machine Learning,Definition,"Bias, Fairness and Ethics",This step examines whether the model produces ...
6,Machine Learning,Definition,Data Collection,"Data collection involves identifying, sourcing..."
7,Machine Learning,Definition,Data Preparation,Data preparation focuses on cleaning and struc...
8,Machine Learning,Definition,Deployment,Deployment is the process of integrating the t...
9,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
10,Machine Learning,Definition,Evaluation,Evaluation measures how well the final model p...
11,Machine Learning,Definition,Feature Engineering,Feature engineering is the process of creating...
12,Machine Learning,Definition,Feature Selection,Feature selection involves choosing a subset o...
13,Machine Learning,Definition,Hyperparameter Tuning,Hyperparameter tuning involves selecting optim...
14,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...


,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
1,Machine Learning,Definition,"Bias, Fairness and Ethics",This step examines whether the model produces ...
2,Machine Learning,Definition,Data Collection,"Data collection involves identifying, sourcing..."
3,Machine Learning,Definition,Data Preparation,Data preparation focuses on cleaning and struc...
4,Machine Learning,Definition,Deployment,Deployment is the process of integrating the t...
...,...,...,...,...
383,Mathematics,Best Linear Unbiased Estimator,Requirements,Homoscedasticity: Variance of the residuals sh...
384,Mathematics,Best Linear Unbiased Estimator,Requirements,Independence: Observations should be independe...
385,Mathematics,Best Linear Unbiased Estimator,Requirements,Linearity: Relationship between Independent an...
386,Mathematics,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity: No perfect linear rel...


In [2]:
test = ml_def[['Category','Categorization','Word','Definition']]
test.head()

,Category,Categorization,Word,Definition
0,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.
1,Best Linear Unbiased Estimator,Requirements,Independence.,Observations should be independent of each oth...
2,Best Linear Unbiased Estimator,Requirements,Linearity,Relationship between Independent and Dependent...
3,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity,No perfect linear relationship between residuals
4,Best Linear Unbiased Estimator,Requirements,Normality of Residuals,Residuals should be normally distributed.


In [3]:
mod_notes = notes.drop(['Category',"Definition"],axis=1).rename(columns={'Categorization':'Category','Word':"Categorization"})
mod_notes.head()

,Category,Categorization
0,Problem Definition,Goal
1,Problem Definition,Approach
2,Problem Definition,Important to Remeber
3,Problem Definition,Lesson Learnt
4,Problem Definition,Algorithm


In [15]:
d_learning_notes[d_learning_notes['Categorization'].fillna('').str.contains('train',case=False)]

,Category,Categorization,Word,Definition
302,Machine Learning,Training,Definition,Training is the process of fitting the model t...
303,Machine Learning,Training,Goal,
304,Machine Learning,Training,Approach,
305,Machine Learning,Training,Important to Remember,
306,Machine Learning,Training,Lesson Learnt,
307,Machine Learning,Training,Algorithm,
308,Machine Learning,Training,Function,Hinge Loss: Loss function used primarily for b...
309,Machine Learning,Training,Function,Huber Loss: Loss function used in regression p...
310,Machine Learning,Training,Procedure,
311,Machine Learning,Training,TBD,


In [21]:
mod_notes

NameError: name 'mod_notes' is not defined

In [4]:
test1 = test.merge(mod_notes,on=['Category','Categorization'],how='left',indicator=True)
test1[test1['_merge']=='left_only']

,Category,Categorization,Word,Definition,_merge
5,Machine Learning,Definition,"Bias, Fairness, and Ethics",This step examines whether the model produces ...,left_only
6,Machine Learning,Definition,Data Collection,"Data collection involves identifying, sourcing...",left_only
7,Machine Learning,Definition,Data Preparation,Data preparation focuses on cleaning and struc...,left_only
8,Machine Learning,Definition,Deployment,Deployment is the process of integrating the t...,left_only
9,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,left_only
10,Machine Learning,Definition,Evaluation,Evaluation measures how well the final model p...,left_only
11,Machine Learning,Definition,Feature Engineering,Feature engineering is the process of creating...,left_only
12,Machine Learning,Definition,Feature Selection,Feature selection involves choosing a subset o...,left_only
13,Machine Learning,Definition,Hyperparameter Tuning,Hyperparameter tuning involves selecting optim...,left_only
14,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...,left_only


In [6]:
ml_notes['Categorization'].unique().tolist()

['Problem Definition',
 'Data Collection',
 'Data Preperation',
 'Feature Engineering',
 'Feature Selection',
 'Model',
 'Training',
 'Hyperparameter Tuning',
 'Validation',
 'Evaluation',
 'Bias, Fairness and Ethics',
 'Model Summarization/ Model Interpretability',
 'Deployment',
 'Monitoring']

In [12]:
d_learning_notes[d_learning_notes['Categorization'].fillna('').str.contains('Training',case=False)]

,Category,Categorization,Word,Definition
302,Machine Learning,Training,Definition,Training is the process of fitting the model t...
303,Machine Learning,Training,Goal,
304,Machine Learning,Training,Approach,
305,Machine Learning,Training,Important to Remember,
306,Machine Learning,Training,Lesson Learnt,
307,Machine Learning,Training,Algorithm,
308,Machine Learning,Training,Function,Hinge Loss: Loss function used primarily for b...
309,Machine Learning,Training,Function,Huber Loss: Loss function used in regression p...
310,Machine Learning,Training,Procedure,
311,Machine Learning,Training,TBD,


In [10]:
d_learning_notes[d_learning_notes['Categorization'].fillna('').str.contains('Bias',case=False)]

,Category,Categorization,Word,Definition
337,Machine Learning,"Bias, Fairness and Ethics",Definition,This step examines whether the model produces ...
338,Machine Learning,"Bias, Fairness and Ethics",Goal,
339,Machine Learning,"Bias, Fairness and Ethics",Approach,
340,Machine Learning,"Bias, Fairness and Ethics",Important to Remeber,
341,Machine Learning,"Bias, Fairness and Ethics",Lesson Learnt,
342,Machine Learning,"Bias, Fairness and Ethics",Algorithm,
343,Machine Learning,"Bias, Fairness and Ethics",Function,
344,Machine Learning,"Bias, Fairness and Ethics",Procedure,
345,Machine Learning,"Bias, Fairness and Ethics",TBD,
378,Mathematics,Best Linear Unbiased Estimator,Definition,Some Definition


In [25]:
notes[notes['Categorization']=='Data Collection;

,Category,Categorization,Word,Definition
8,Machine Learning,Data Collection,Goal,Need this to Pull from Somewhere Else.
9,Machine Learning,Data Collection,Approach,
10,Machine Learning,Data Collection,Important to Remeber,
11,Machine Learning,Data Collection,Lesson Learnt,
12,Machine Learning,Data Collection,Algorithm,
13,Machine Learning,Data Collection,Function,
14,Machine Learning,Data Collection,Procedure,
15,Machine Learning,Data Collection,TBD,


In [20]:



d_learning_notes = final_dataset_for_markdown(notes,definitions)
d_learning_notes

,Category,Categorization,Definition,Word
0,Machine Learning,Problem Definition,Problem definition clearly states the business...,Definition
1,Machine Learning,Data Collection,"Data collection involves identifying, sourcing...",Definition
2,Machine Learning,Data Preperation,Not Defined,Definition
3,Machine Learning,Feature Engineering,Feature engineering is the process of creating...,Definition
4,Machine Learning,Feature Selection,Feature selection involves choosing a subset o...,Definition
5,Machine Learning,Model,The model is the mathematical or algorithmic s...,Definition
6,Machine Learning,Training,Training is the process of fitting the model t...,Definition
7,Machine Learning,Hyperparameter Tuning,Hyperparameter tuning involves selecting optim...,Definition
8,Machine Learning,Validation,Process which evaluates a trained machine-lear...,Definition
9,Machine Learning,Evaluation,Evaluation measures how well the final model p...,Definition


In [23]:
notes[notes['Word']=='Data Collection']

,Category,Categorization,Word,Definition


,Category,Categorization,Word,Definition,_merge
5,"Bias, Fairness, and Ethics",Definition,"Bias, Fairness, and Ethics",This step examines whether the model produces ...,left_only
6,Data Collection,Definition,Data Collection,"Data collection involves identifying, sourcing...",left_only
7,Data Preparation,Definition,Data Preparation,Data preparation focuses on cleaning and struc...,left_only
8,Deployment,Definition,Deployment,Deployment is the process of integrating the t...,left_only
9,Evaluation,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,left_only
10,Evaluation,Definition,Evaluation,Evaluation measures how well the final model p...,left_only
11,Feature Engineering,Definition,Feature Engineering,Feature engineering is the process of creating...,left_only
12,Feature Selection,Definition,Feature Selection,Feature selection involves choosing a subset o...,left_only
13,Hyperparameter Tuning,Definition,Hyperparameter Tuning,Hyperparameter tuning involves selecting optim...,left_only
14,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...,left_only


In [8]:


d_learning_notes = final_dataset_for_markdown_v2(notes,definitions)  
d_learning_notes

#d_learning_notes = final_dataset_for_markdown(notes,definitions)


,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
1,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
2,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
4,Machine Learning,Definition,Cross Validation,Model evaluation technique used to assess how ...
...,...,...,...,...
370,Mathematics,Best Linear Unbiased Estimator,Requirements,Homoscedasticity: Variance of the residuals sh...
371,Mathematics,Best Linear Unbiased Estimator,Requirements,Independence.: Observations should be independ...
372,Mathematics,Best Linear Unbiased Estimator,Requirements,Linearity: Relationship between Independent an...
373,Mathematics,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity: No perfect linear rel...


In [11]:
d_learning_notes['Categorization'].value_counts()

Categorization
Model                                          228
Definition                                      18
Validation                                      12
Feature Selection                               11
Training                                        10
Problem Definition                               9
Data Collection                                  9
Data Preperation                                 9
Feature Engineering                              9
Hyperparameter Tuning                            9
Evaluation                                       9
Bias, Fairness and Ethics                        9
Model Summarization/ Model Interpretability      9
Deployment                                       9
Monitoring                                       9
Best Linear Unbiased Estimator                   6
Name: count, dtype: int64

In [24]:
def search_df_word(df,word,columns=None,case_sensitive=False,print_=True):
    '''
    
    
    '''

    # If User does not define which columns they wish to search, search all.
    if not columns:
        columns = df.columns.tolist()

    final_dict = {}
    final_df = pd.DataFrame()

    for column in columns:
        match_df = df[df[column].fillna('').str.contains(word,case=case_sensitive)]
        if len(match_df)==0:
            final_dict[column] = 'No Matches'
        else:
            final_word = ""
            for word in match_df[column]:
                final_word += word + ","
        
            final_dict[column] = final_word
            final_df = pd.concat([final_df,match_df])

    if print_:
        try:
            print(final_df)
        except:
            pass
            
    return final_df, final_dict

#final_df,final_dict = search_df_word(d_learning_notes,'regularization')
final_df,final_dict = search_df_word(d_learning_notes,'validation')

    

             Category Categorization                   Word  \
307  Machine Learning     Validation             Definition   
308  Machine Learning     Validation                   Goal   
309  Machine Learning     Validation               Approach   
310  Machine Learning     Validation  Important to Remember   
311  Machine Learning     Validation          Lesson Learnt   
312  Machine Learning     Validation              Algorithm   
313  Machine Learning     Validation               Function   
314  Machine Learning     Validation              Procedure   
315  Machine Learning     Validation              Procedure   
316  Machine Learning     Validation              Procedure   
317  Machine Learning     Validation                    TBD   
5    Machine Learning     Definition       Cross Validation   
315  Machine Learning     Validation              Procedure   
316  Machine Learning     Validation              Procedure   

                                            Definition

### Examples of How Information is Included

In [87]:
def d_learning_notes_guide(example_word_dict={},
                           df=d_learning_notes,
                           notes=notes,
                           definitions=definitions):

    if len(example_word_dict)==0:
        example_word_dict= {'Category':'Machine Learning',
                            'Categorization':'Model',
                            'Word':'Bias - Variance Trade Off',
                            'Definition':'Linear Regression'}
    
    for key,word in example_word_dict.items():
        print('\n############################### New Word ###############################\n')
        print(f'Word: {word}')
        print(f'Word Classification: {key}')
    
        for key,data_set in {'notes':notes,'definitions':definitions,'d_learning_notes':d_learning_notes}.items():
    
            print(f"\nInclusion in {key}:")
            for column in ['Category','Categorization','Word','Definition']:
                try:
                    temp = data_set[data_set[column]==word][['Category','Categorization','Word','Definition']].copy()
                except:
                    temp = data_set[data_set[column]==word].copy()
                
                print(f'\nAs Word in {column}')
                
                if len(temp)>0:    
                    display(temp.head(10))
                else:
                    print(f'No Records Found')
                
d_learning_notes_guide()


############################### New Word ###############################

Word: Machine Learning
Word Classification: Category

Inclusion in notes:

As Word in Category


,Category,Categorization,Word,Definition
0,Machine Learning,Problem Definition,Goal,Need this to Pull from Somewhere Else.
1,Machine Learning,Problem Definition,Approach,
2,Machine Learning,Problem Definition,Important to Remeber,
3,Machine Learning,Problem Definition,Lesson Learnt,
4,Machine Learning,Problem Definition,Algorithm,
5,Machine Learning,Problem Definition,Function,
6,Machine Learning,Problem Definition,Procedure,
7,Machine Learning,Problem Definition,TBD,
8,Machine Learning,Data Collection,Goal,Need this to Pull from Somewhere Else.
9,Machine Learning,Data Collection,Approach,



As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found

Inclusion in definitions:

As Word in Category


,Category,Categorization,Word,Definition
0,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
1,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
2,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
4,Machine Learning,Definition,Cross Validation,Model evaluation technique used to assess how ...
5,Machine Learning,Definition,Curse of Dimensionality,"As dimensionality grows, data points become in..."
6,Machine Learning,Definition,Deep Learning,Deep learning is a subset of machine learning ...
7,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
8,Machine Learning,Definition,Optimization,Optimization is the process of finding the bes...
9,Machine Learning,Definition,Reinforcement Learning,Reinforcement learning trains an agent to make...



As Word in Categorization
No Records Found

As Word in Word


...


As Word in Definition
No Records Found

Inclusion in d_learning_notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word


,Category,Categorization,Word,Definition
4,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...



As Word in Definition
No Records Found

############################### New Word ###############################

Word: Linear Regression
Word Classification: Definition

Inclusion in notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found

Inclusion in definitions:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word


,Category,Categorization,Word,Definition
109,Model,Algorithm,Linear Regression,Models the relationship between an independent...



As Word in Definition
No Records Found

Inclusion in d_learning_notes:

As Word in Category
No Records Found

As Word in Categorization
No Records Found

As Word in Word
No Records Found

As Word in Definition
No Records Found


In [ ]:
def definition_dq(df,exclude_rule1=['Machine Learning']):
    
    '''
    
    
    '''
    
    df = df.copy()
    
    # Rule 1: Word and Category Can not Equal
    df['Rule1'] = np.where((df['Category']==df['Word'])&(~df['Category'].isin(exclude_rule1)),1,0)
    
    # Rule 2: There should be No Duplication in Word Column
    df['Rule2'] = df.groupby('Word').transform('size')
    df['Rule2'] = np.where(df['Rule2']>1,1,0)

    col_list =  [x for x in df.columns if x.find('Rule')!=-1]
    df['Violation'] = df[col_list].sum(axis=1)
    df['Violation'] = np.where(df['Violation']>0,1,0)
        
    return df.drop(['Notes','Link','Image','Markdown Equation','Dataset Size','Learning Type','Algorithm Class'],axis=1)

test = definition_dq(definitions)

test[test['Violation']==1]